In [1]:
import os
import numpy as np
import re
import pickle
import sys

pd.set_option('display.float_format', lambda x: '%.6f' % x)

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from ProcessEventsFunctions import *
# from Convert_to_Profiles_Functions import *
# from Get_Events_Functions import *

In [2]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

### Check number of years covered by each time period

In [3]:
len(list(range(2001,2020)))
len(list(range(2061,2080)))

19

### Check if there is the same number of files in the folder containing all events

For each gauge, count the number of files (part0 in name, so not including the CAM)

Answer = 19 * 1291 * 7 = 171703

In [25]:
# gauge_num=11
# time_period='Present'
# em='bc005'
# files  = os.listdir(home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet")
# # files = [item for item in files if 'part1' not in item]
# len(files)

# np.sort(files)

In [70]:
time_period='Present'
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
# ems_present = ['bc016']
cams_dict_present ={}
events_dict_present={}
for em in ems_present:
    cams_thisem = 0
    events_thisem = 0    
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}_messed/{gauge_num}/WholeYear/EventSet"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)
            if len([item for item in files if 'part1' not in item])!= 133:
                print(em, gauge_num, len([item for item in files if 'part1' not in item]))
            for item in files:
                if 'part1' in item:
                    cams_thisem=cams_thisem+1
                elif 'part0' in item:
                    events_thisem=events_thisem+1                          
            cams_dict_present[em] = cams_thisem
            events_dict_present[em] = events_thisem         
events_dict_present            

bc005 3 132
bc005 11 131
bc005 13 132
bc005 29 132
bc005 34 132
bc005 41 132
bc005 46 132
bc005 58 132
bc005 61 132
bc005 66 132
bc005 68 132
bc005 104 131
bc005 107 132
bc005 109 132
bc005 191 132
bc005 225 132
bc005 245 132
bc005 251 131
bc005 293 132
bc005 296 132
bc005 297 131
bc005 298 131
bc005 299 132
bc005 300 131
bc005 301 132
bc005 303 131
bc005 304 132
bc005 306 132
bc005 310 131
bc005 312 132
bc005 318 132
bc005 319 132
bc005 322 132
bc005 324 131
bc005 325 131
bc005 326 131
bc005 328 130
bc005 330 132
bc005 331 132
bc005 335 132
bc005 336 132
bc005 337 132
bc005 344 132
bc005 345 132
bc005 346 132
bc005 347 132
bc005 348 131
bc005 352 131
bc005 354 132
bc005 366 132
bc005 367 131
bc005 371 131
bc005 373 132
bc005 374 132
bc005 375 132
bc005 380 131
bc005 382 132
bc005 383 132
bc005 384 132
bc005 385 132
bc005 386 132
bc005 387 132
bc005 388 132
bc005 389 132
bc005 391 132
bc005 393 130
bc005 395 132
bc005 398 131
bc005 400 132
bc005 404 132
bc005 408 131
bc005 409 132
bc00

bc005 1267 131
bc005 1268 132
bc005 1270 132
bc005 1271 132
bc005 1273 132
bc005 1275 131
bc005 1276 130
bc005 1280 131
bc005 1281 132
bc005 1286 132
bc005 1287 131
bc005 1288 131
bc005 1289 131
bc005 1290 132
bc005 1292 132
bc005 1293 132


KeyboardInterrupt: 

In [35]:
time_period='Future'
ems_future = ['bb195', 'bb192', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211']
ems_future = ['bb204', 'bb198']

cams_dict_future={}
events_dict_future={}
for em in ems_future:
    cams_thisem = 0
    events_thisem = 0
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet/"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)
            if len([item for item in files if 'part1' not in item])!= 133:
                print(em, gauge_num, len([item for item in files if 'part1' not in item]))
            for item in files:
                if 'part1' in item:
                    cams_thisem=cams_thisem+1
                elif 'part0' in item:
                    events_thisem=events_thisem+1                    
            cams_dict_future[em] = cams_thisem
            events_dict_future[em] = events_thisem       
events_dict_future            

{'bb204': 171703, 'bb198': 171703}

### Check if there are enough files for every duration
Counting just events with 'part0' otherwise will be different due to CAM - should always be 19

In [51]:
durations = ['0.5', '1', '2', '3', '6', '12', '24']

In [58]:
time_period = 'Present'
for em in ['bc017']:
    print(em)
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            # print(f"gauge num {gauge_num}")
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet/"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)
            for duration in durations:
                count_starts = sum(s.startswith(f"{duration}hrs") and 'part0' in s for s in files)
                # print(duration, count_starts)
                if count_starts != 19:
                    print('broken', count_starts, duration)
                    break

bc017


## Check number of events when combined into the dictionary

In [94]:
def count_number_events(em, time_period):
    print(em)
    cams_thisem = 0
    events_thisem = 0
    for gauge_num in range(0, 1294):
        if gauge_num not in [444, 827, 888]:
            indy_events_fp = home_dir2 + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/EventSet/"
            files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
            files = np.sort(files)

            for item in files:
                if 'part1' in item:
                    cams_thisem=cams_thisem+1
                elif 'part0' in item:
                    events_thisem=events_thisem+1   
                        
    return events_thisem, cams_thisem, cams_thisem+events_thisem

In [76]:
events_props_dict_present = []
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
for em in ems_present:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_props_dict_{em}.pickle", 'rb') as handle:
        events_props_dict = pickle.load(handle)    
    events_props_dict_present = events_props_dict_present + events_props_dict
## Join into one dataframe    
present_allevents = pd.DataFrame(events_props_dict_present)
present_allevents['Climate'] = 'Present'    

events_props_dict_future = []
ems_future = ['bb195', 'bb192', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211']
for em in ems_future:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_{em}.pickle", 'rb') as handle:
        events_props_dict = pickle.load(handle)    
    events_props_dict_future = events_props_dict_future + events_props_dict
## Join into one dataframe    
future_allevents = pd.DataFrame(events_props_dict_future)
future_allevents['Climate'] = 'Future'        

### Total number of events
Will be slightly different for each because of deletion of duplicates not always being the same

In [44]:
ems_future = ['bb192', 'bb195', 'bb198', 'bb201', 'bb204','bb208' ,'bb211','bb216', 'bb219','bb222','bb225']
for em in ems_future:
    # nun = count_number_events(em, 'Future')
    # print(nun[2])
    # This is the TOTAL number of events
    print(len([d for d in events_props_dict_future if d.get('em') == em]))

89106
89097
92280
89686
91815
89561
91231
89210
89313
92102
90905


## Number of events for each EM
Different number of events each EM, because its all events, no duplicates, and number of events which are the AMAX for different durations will vary

In [137]:
ems_future = ['bb192', 'bb195', 'bb198', 'bb201', 'bb204','bb208' ,'bb211','bb216', 'bb219','bb222','bb225']
for em in ems_future:
    this_em = future_allevents[future_allevents['em']==em]
    print(len(this_em))

96633
96367
98418
95788
98330
97190
98733
96719
97057
98295
97822


## Checking the number of part0 events
Get data for one em.  
Get data for one duration.  
Get data for one gauge, check if there are 19 events with part 0 in the string

Matching rows sometimes has more than one entry per year, for the same duration. This is presumably because of CAM. However it's confusing because in that case I might have expected the filename to be Part1, but its not. 

In [71]:
ems_future = ['bc006']
for em in ems_future:
    this_em = present_allevents[present_allevents['em']==em]
    print(len(this_em))

90541


In [72]:
# ems_future = ['bb195']
# for em in ems_future:
#     this_em = present_allevents[present_allevents['em']==em]
#     print(len(this_em))

In [73]:
duration =24
future_allevents_6h = this_em[this_em['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
# future_allevents_6h.sort_values(by=['gauge_num', 'year'], ascending=[True, True])

for gauge_num in range(0,1294):
    this_gauge= future_allevents_6h[future_allevents_6h['gauge_num']==gauge_num]
    matching_rows = this_gauge[this_gauge["filename"].str.contains('part0', na=False)]
    if len(matching_rows) != 19:
        print(gauge_num)
        break
#     print(len(matching_rows))

5


In [78]:
this_gauge= future_allevents_6h[future_allevents_6h['gauge_num']==7]
matching_rows.sort_values(by=['gauge_num', 'year'], ascending=[True, True])

,season,duration,DurationRange_personalised_allems,DurationRange_notpersonalised,DurationRange_simple,year,month,Volume,max_intensity,max_quintile_molly,...,D50_index,theta,D50,com,dur_for_which_this_is_amax,gauge_num,area,em,filename,Climate
106608,Winter,9.5,2.10-6.45 hr,6.45-19.25 hr,4-12hr,2001,5,44.686627,33.595410,1,...,1.0,2.167505,5.555556,0.250011,"[12, 24]",5,SE,bc006,12hrs_2001_v2_part0.csv,Present
106628,Winter,1.5,0.25-2.10 hr,0.25-2.10 hr,<4hr,2001,5,1.790528,2.264113,3,...,1.0,2.150303,50.000000,0.344890,24,5,SE,bc006,24hrs_2001_v2_part0.csv,Present
106629,Summer,20.0,6.45-19.25 hr,19.25+ hr,12hr+,2002,7,73.020156,51.925866,5,...,37.0,3.440485,94.871795,0.890099,24,5,SE,bc006,24hrs_2002_v2_part0.csv,Present
106630,Winter,17.5,6.45-19.25 hr,6.45-19.25 hr,12hr+,2003,1,33.647423,6.125627,3,...,14.0,0.292441,41.176471,0.423243,24,5,SE,bc006,24hrs_2003_v2_part0.csv,Present
106631,Winter,20.0,6.45-19.25 hr,19.25+ hr,12hr+,2004,1,30.140281,8.079622,2,...,12.0,0.430061,30.769231,0.358741,24,5,SE,bc006,24hrs_2004_v2_part0.csv,Present
106593,Summer,29.5,19.25+ hr,19.25+ hr,12hr+,2005,8,50.253046,13.101843,1,...,21.0,4.025367,36.206897,0.434776,"[0.5, 1, 24, 2, 3]",5,SE,bc006,0.5hrs_2005_v2_part0.csv,Present
106632,Summer,21.5,6.45-19.25 hr,19.25+ hr,12hr+,2006,7,44.613156,12.333244,5,...,38.0,3.216853,90.476190,0.828009,24,5,SE,bc006,24hrs_2006_v2_part0.csv,Present
106595,Summer,1.5,0.25-2.10 hr,0.25-2.10 hr,<4hr,2007,7,56.692548,74.040220,3,...,1.0,3.492092,50.000000,0.255028,"[0.5, 12, 1, 24, 2, 3, 6]",5,SE,bc006,0.5hrs_2007_v2_part0.csv,Present
106633,Summer,1.0,NaN,0.25-2.10 hr,<4hr,2007,7,0.489359,0.637650,1,...,0.0,3.474890,0.000000,0.174242,24,5,SE,bc006,24hrs_2007_v2_part0.csv,Present
106634,Winter,23.5,19.25+ hr,19.25+ hr,12hr+,2008,3,23.878681,5.624438,5,...,28.0,1.014943,60.869565,1.411856,24,5,SE,bc006,24hrs_2008_v2_part0.csv,Present


In [60]:
this_em.sort_values(by=['gauge_num', 'year'], ascending=[True, True])

,season,duration,DurationRange_personalised_allems,DurationRange_notpersonalised,DurationRange_simple,year,month,Volume,max_intensity,max_quintile_molly,...,D50_index,theta,D50,com,dur_for_which_this_is_amax,gauge_num,area,em,filename,Climate
0,Summer,6.5,2.10-6.45 hr,6.45-19.25 hr,4-12hr,2001,7,13.278191,14.863884,5,...,11.0,3.526497,91.666667,0.596839,0.5,0,ME,bc005,0.5hrs_2001_v2_part0.csv,Present
19,Winter,8.5,2.10-6.45 hr,6.45-19.25 hr,4-12hr,2001,11,36.565513,10.505643,2,...,6.0,5.263942,37.500000,0.409806,"[12, 1, 2, 3, 6]",0,ME,bc005,12hrs_2001_v2_part0.csv,Present
40,Winter,22.0,6.45-19.25 hr,19.25+ hr,12hr+,2001,11,41.910386,10.505643,1,...,7.0,5.263942,16.279070,0.246878,24,0,ME,bc005,24hrs_2001_v2_part0.csv,Present
1,Summer,2.5,0.25-2.10 hr,2.10-6.45 hr,<4hr,2002,8,10.344041,9.528421,2,...,1.0,4.076974,25.000000,0.364555,"[0.5, 1, 2]",0,ME,bc005,0.5hrs_2002_v2_part0.csv,Present
20,Winter,15.5,6.45-19.25 hr,6.45-19.25 hr,12hr+,2002,4,22.630407,5.708532,2,...,12.0,1.737445,40.000000,0.483790,12,0,ME,bc005,12hrs_2002_v2_part0.csv,Present
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106212,Summer,6.5,2.10-6.45 hr,6.45-19.25 hr,4-12hr,2018,6,25.018182,17.014418,2,...,4.0,2.631971,33.333333,0.347768,"[2, 3]",1293,NE,bc005,2hrs_2018_v2_part0.csv,Present
106159,Summer,1.5,0.25-2.10 hr,0.25-2.10 hr,<4hr,2019,6,13.425980,24.169712,3,...,1.0,2.649173,50.000000,0.325425,0.5,1293,NE,bc005,0.5hrs_2019_v2_part0.csv,Present
106177,Winter,17.0,6.45-19.25 hr,6.45-19.25 hr,12hr+,2019,12,29.757842,3.230366,5,...,19.0,5.952039,57.575758,0.550930,12,1293,NE,bc005,12hrs_2019_v2_part0.csv,Present
106184,Summer,5.0,0.25-2.10 hr,2.10-6.45 hr,4-12hr,2019,8,18.948213,15.029365,4,...,7.0,3.801736,77.777778,0.638159,"[1, 2, 3, 6]",1293,NE,bc005,1hrs_2019_v2_part0.csv,Present


In [63]:
matching_rows.sort_values(by=['gauge_num', 'year'], ascending=[True, True])

,season,duration,DurationRange_personalised_allems,DurationRange_notpersonalised,DurationRange_simple,year,month,Volume,max_intensity,max_quintile_molly,...,D50_index,theta,D50,com,dur_for_which_this_is_amax,gauge_num,area,em,filename,Climate
219,Winter,24.0,19.25+ hr,19.25+ hr,12hr+,2001,1,38.882600,3.562829,1,...,17.0,0.361251,36.170213,0.408796,24,2,SE,bc005,24hrs_2001_v2_part0.csv,Present
220,Winter,23.5,19.25+ hr,19.25+ hr,12hr+,2002,11,40.311569,4.735188,2,...,17.0,5.607990,36.956522,0.402219,24,2,SE,bc005,24hrs_2002_v2_part0.csv,Present
221,Winter,24.0,19.25+ hr,19.25+ hr,12hr+,2003,1,28.790135,6.841623,1,...,10.0,0.481668,21.276596,0.369805,24,2,SE,bc005,24hrs_2003_v2_part0.csv,Present
174,Summer,2.5,0.25-2.10 hr,2.10-6.45 hr,<4hr,2004,7,43.481478,38.833794,2,...,1.0,3.578104,25.000000,0.250076,"[0.5, 12, 1, 24, 24, 2, 3, 6]",2,SE,bc005,0.5hrs_2004_v2_part0.csv,Present
222,Winter,29.0,19.25+ hr,19.25+ hr,12hr+,2005,11,38.293258,11.158742,5,...,26.0,5.573585,45.614035,0.549723,24,2,SE,bc005,24hrs_2005_v2_part0.csv,Present
223,Summer,23.0,19.25+ hr,19.25+ hr,12hr+,2006,5,36.972702,7.605535,4,...,29.0,2.459947,64.444444,0.576910,24,2,SE,bc005,24hrs_2006_v2_part0.csv,Present
224,Summer,25.5,19.25+ hr,19.25+ hr,12hr+,2007,9,40.311665,12.125032,2,...,15.0,4.386618,30.000000,0.309287,24,2,SE,bc005,24hrs_2007_v2_part0.csv,Present
226,Winter,51.5,19.25+ hr,19.25+ hr,12hr+,2008,11,41.859240,7.768946,4,...,67.0,5.384359,65.686275,0.556983,24,2,SE,bc005,24hrs_2008_v2_part0.csv,Present
227,Winter,22.5,6.45-19.25 hr,19.25+ hr,12hr+,2009,10,33.887097,11.815922,3,...,18.0,4.747869,40.909091,0.323555,24,2,SE,bc005,24hrs_2009_v2_part0.csv,Present
229,Winter,24.5,19.25+ hr,19.25+ hr,12hr+,2010,10,38.100744,10.889741,2,...,14.0,5.143525,29.166667,0.345290,24,2,SE,bc005,24hrs_2010_v2_part0.csv,Present


In [148]:
### But then if we filter to just show events which are AMAX producing for a certain duration, the numbers should
# be the same again
ems_future = ['bb192', 'bb195']
for em in ems_future:
    this_em = future_allevents_test[future_allevents_test['em']==em]
    ## Get just the events for this duration
    future_allevents_6h = this_em[this_em['dur_for_which_this_is_amax'].apply(
        lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
    # Keep only the part0 events
    matching_rows = future_allevents_6h[future_allevents_6h["filename"].str.contains('part0', na=False)]
    print(duration, len(future_allevents_6h), len(matching_rows))

24 215 194
24 243 202


In [136]:
em='bb195'
duration = '6'

# Get just the events for this em
this_em = future_allevents[future_allevents['em']==em]

## Get just the events for this duration
future_allevents_6h = this_em[this_em['dur_for_which_this_is_amax'].apply(
    lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]

# Keep only the part0 events
matching_rows = future_allevents_6h[future_allevents_6h["filename"].str.contains('part0', na=False)]

print(duration, len(future_allevents_6h), len(matching_rows))

6 24548 24281


In [138]:
ems_future = ['bb195']
for em in ems_future:
    this_em = future_allevents[future_allevents['em']==em]
    for duration in durations:
        df_long_this_dur = this_em[this_em['dur_for_which_this_is_amax'].apply(
            lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
        matching_rows = df_long_this_dur[df_long_this_dur["filename"].str.contains('part0', na=False)]
        print(duration, len(df_long_this_dur), len(matching_rows))
    

0.5 24529 24529
1 24529 24517
2 24529 24479
3 24529 24410
6 24548 24281
12 25058 24648
24 28546 25351


In [100]:
ems_future = ['bb192', 'bb195', 'bb198', 'bb201', 'bb204','bb208' ,'bb211','bb216', 'bb219','bb222','bb225']
for em in ems_future:
    #nun = count_number_events(em, 'Future')
    #print(nun[2])
    # This is the TOTAL number of events
    # print(len([d for d in events_props_dict_future if d.get('em') == em]))
    for duration in durations:
        df_long_this_dur = future[future['dur_for_which_this_is_amax'].apply(
            lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
        print(duration, len(df_long_this_dur))
    

bb192
177310
96633
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb195
177139
96367
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb198
175702
98418
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb201
175703
95788
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb204
176778
98330
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb208
178247
97190
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb211
177079
98733
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb216
177591
96719
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb219
177942
97057
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb222
175521
98295
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580
bb225
176600
97822
0.5 269815
1 269789
2 269785
3 269782
6 269938
12 275062
24 312580


In [16]:
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
for em in ems_present:
    nun = count_number_events(em, 'Present')
    print(f"not incl CAMS {nun[0]}")
    print(nun[2])
    print(len([d for d in events_props_dict_present if d.get('em') == em]))

bc005
not incl CAMS 171703
125321
bc006
not incl CAMS 171703
120674
bc007
not incl CAMS 171703
118921
bc009
not incl CAMS 171703
115411
bc010
not incl CAMS 171703
117272
bc011
not incl CAMS 171703
118085
bc012
not incl CAMS 171703
120344
bc013
not incl CAMS 171703
119710
bc015
not incl CAMS 171703
120124
bc016
not incl CAMS 171703
120683
bc017
not incl CAMS 171703
115840
bc018
not incl CAMS 171703
120812
